## LDA Preprocessing 1
# Artifact Removal and Stopword Selection
This notebook takes the clean JSON files for each article and does some preprocessing to obtain a text that we can analyze using LDA.

Specifically, we do:
* Artifact removal
* Stopword selection

In the next notebook we will do:
* Punctuation removal
* Lemmatization

In [1]:
import json
import re

We will also use some utility functions we defined in the `utils/` folder:
* `loadCorpusList(path)`: Loads the corpus as a list of `Article` objects (see `utils/Article.py`). This will allow us to save the clean text per document into the same JSON file with the metadata included.
* `saveCorpus(path)`: Saves the articles in JSON format in their current state. Useful when we want to append information to our clean JSON files.

In [2]:
import os
import sys 

# Jupyter Notebooks are not good at handling relative imports.
# Best solution (not great practice) is to add the project's path
# to the module loading paths of sys.

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.loaders import loadCorpusList, saveCorpus

## Loading the corpus

In [3]:
corpusPath = '../data/clean_json'

corpusList = loadCorpusList(corpusPath)

In [4]:
corpusList[0].title

'Kasahara, Javier. “Una relectura leibniziana al mecanicismo.”'

We will only work with documents in Spanish. Hence, let's replace `corpusList` with only the articles that are in Spanish.

In [5]:
corpusList = [doc for doc in corpusList if doc.lang == 'es']

This leaves is with 554 articles.

In [6]:
len(corpusList)

554

(Este paso estaba en el otro notebook y siento que es importante pero no conozco bien la justificación.)

In [7]:
for doc in corpusList:
    doc.cleanText = doc.text.encode("utf8").decode("utf8")

## Artifact removal

There are some artifacts included in the text that are produced by HTML processing (or in the future because of how PDF files store text).

Let's start by removing numbers and some special characters such as newline characters (`\n`). We will keep normal punctuation for now as that might help SpaCy when we do lemmatization.

In [8]:
for doc in corpusList:
    doc.cleanText = re.sub('\d|\n',' ', doc.text)

We can detect some of these artifacts by looking for non-alphanumeric characters between alphanumeric characters (e.g. `"ar-gument"`, `"ar\xadgument"`).

In [9]:
artifacts = re.compile('\w+[^a-zA-ZáéíóúÁÉÍÓÚñÑüÜ\d\s:]\w+')

In [10]:
[re.findall(artifacts, doc.text) for doc in corpusList][0]

['35-48',
 'mecanicismo,1',
 'mente-cuerpo',
 'otro.2',
 'primero.3',
 '148-149',
 '125-194',
 'montenegrop.nicolas',
 'urosario.edu']

One common artifact is the hex `\xad` for the soft hyphen which is used to break lines. We can remove it easily.

In [11]:
for doc in corpusList:
    doc.cleanText = re.sub('\\xad',' ', doc.cleanText)

We can save the corpus for now.

In [12]:
saveCorpus('../data/clean_json', corpusList)

## Stopword Removal
Stopword removal is perhaps the most difficult part of preprocessing. There are two challenges to meet:
* Some stopword lists such as the one included in NLTK for Spanish are too weak and do not filter many stopwords.
* Other stopword lists are too inclusive and can eliminate words that are meaningful in philosophy (e.g. 'verdadero', true). 
It is important to note that stopwords are very context-sensitive. A word in one context may provide little meaning (hence counting as a stopword) while in other contexts it may provide lots of information.

To tackle these challenges, we will first to an initial filtering with NTLK's list. This will leave many stopwords in the text, but will reduce the size of each text considerably. Then we will compare the text with a stronger list of stopwords (source). We will see which words are both the text and the stronger stopwords list. We will inspect these lists manually and extract a list of protected words. We will iterate over this process a number of times. Once we have a robust list of protected words, we will concatenate NLTK's stopwords list with the stronger one and eliminate the protected words from it. This will provide a final (hopefully middle ground) stopword list with which to continue.

In [13]:
from nltk.corpus import stopwords

stopwords_weak = stopwords.words('spanish')

In [14]:
import requests

r = requests.get('https://raw.githubusercontent.com/stopwords-iso/stopwords-es/master/stopwords-es.txt')
stopwords_strong = r.text.split('\n')

In [15]:
docWords = []
for doc in corpusList:
    docWords += [word for word in re.findall('\w+', doc.cleanText) if word not in stopwords_weak]

In [16]:
docWords[:10]

['Documento',
 'título',
 'Kasahara',
 'Javier',
 'Una',
 'relectura',
 'leibniziana',
 'mecanicismo',
 'Eidos',
 'Kasahara']

In [17]:
from collections import Counter

docs_and_stopwords = Counter([word for word in docWords if word in stopwords_strong])

In [18]:
docs_and_stopwords.most_common(10)

[('ser', 11711),
 ('puede', 7417),
 ('si', 6667),
 ('sino', 6335),
 ('mismo', 5459),
 ('bien', 5069),
 ('modo', 4206),
 ('decir', 4033),
 ('manera', 3743),
 ('solo', 3667)]

Already in the first 10 most common words in both the documents and the strong list of stopwords we find words that in philosophy are quite meaningful:
* 'ser': being
* 'bien': good
* 'modo': mode

We will start saving those words and eliminating them from the stronger list of stopwords. Then we will repeat the process of selecting the words that are in both lists and see which words are common. By iterating over this process a couple of times, we will get a list of protected words.

In [19]:
protectedWords = [
    'ser',
    'bien',
    'modo'
]

In [25]:
stopwords_strong = [word for word in stopwords_strong if word not in protectedWords]
docs_and_stopwords = Counter([word for word in docWords if word in stopwords_strong])

In [26]:
docs_and_stopwords.most_common(100)

[('primero', 877),
 ('última', 853),
 ('quiere', 849),
 ('anterior', 837),
 ('ninguna', 801),
 ('siguiente', 796),
 ('cierta', 792),
 ('demás', 786),
 ('ambos', 771),
 ('nunca', 753),
 ('ningún', 748),
 ('gran', 745),
 ('sigue', 740),
 ('mismos', 713),
 ('deben', 711),
 ('nueva', 696),
 ('considera', 696),
 ('aquella', 689),
 ('allí', 683),
 ('luego', 680),
 ('algún', 675),
 ('c', 662),
 ('ahí', 662),
 ('aquellos', 652),
 ('lleva', 636),
 ('final', 617),
 ('aquel', 617),
 ('buena', 612),
 ('tampoco', 610),
 ('habla', 610),
 ('pasado', 606),
 ('pesar', 604),
 ('va', 588),
 ('siendo', 587),
 ('i', 579),
 ('d', 579),
 ('ciertas', 577),
 ('aun', 575),
 ('x', 569),
 ('ciertos', 561),
 ('supuesto', 559),
 ('después', 556),
 ('hacen', 544),
 ('largo', 532),
 ('igual', 523),
 ('veces', 519),
 ('mal', 514),
 ('propias', 479),
 ('l', 476),
 ('mismas', 472),
 ('aquellas', 467),
 ('solamente', 463),
 ('muchas', 455),
 ('propios', 449),
 ('creo', 418),
 ('poner', 414),
 ('todavía', 412),
 ('adelant

In [27]:
protectedWords += [
    'parte',
    'posible',
    'lugar',
    'hecho',
    'poder',
    'verdad',
    'cosas',
    'general',
    'fin',
    'trabajo',
    'cierto',
    'uso',
    'dado',
    'diferentes',
    'verdadero',
    'verdadera',
    'existe',
    'valor',
    'realizar',
    'existen',
    'conocer',
    'diferente'
]

protectedWords = list(set(protectedWords))

After a couple of times to make the process less complex, once we are sure of a set of words, we can eliminate those from the list of document words and go back and repeat the process a couple more times.

In [28]:
stopwordsToRemove = [word[0] for word in docs_and_stopwords.most_common(100)]
docWords = [word for word in docWords if word not in stopwordsToRemove]

Once we have a robust set of words we can save both the final stopword list and the protected words list.

In [29]:
stopwords_final = list(set(stopwords_weak + stopwords_strong))
with open('wordlists/stopwords.txt', 'w') as fp:
    fp.write('\n'.join(stopwords_final))

with open('wordlists/protectedWords.txt', 'w') as fp:
    fp.write('\n'.join(protectedWords))